## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,-16.71,91,93,5.93,overcast clouds
1,1,Albany,US,42.6001,-73.9662,23.74,53,100,5.99,overcast clouds
2,2,Ahipara,NZ,-35.1667,173.1667,68.63,57,86,8.86,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,48.11,45,9,18.75,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,78.62,71,66,12.82,broken clouds


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input("Please enter the minimum temperature you would like to experience for your trip? "))
maximum_temp = float(input("Please enter the maximum temperature you would like to experience for your trip? "))

Please enter the minimum temperature you would like to experience for your trip? 72
Please enter the maximum temperature you would like to experience for your trip? 82


In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= maximum_temp) & (city_data_df["Max Temp"] >= minimum_temp)]
ideal_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Rikitea,PF,-23.1203,-134.9692,78.62,71,66,12.82,broken clouds
10,10,Avarua,CK,-21.2078,-159.7750,80.65,78,40,2.30,scattered clouds
12,12,Hilo,US,19.7297,-155.0900,78.24,85,13,5.01,few clouds
15,15,Dakar,SN,14.6937,-17.4441,72.48,67,68,19.13,broken clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,78.39,88,42,12.03,light rain
...,...,...,...,...,...,...,...,...,...,...
732,732,Richards Bay,ZA,-28.7830,32.0377,75.79,92,76,5.79,broken clouds
733,733,Maraa,BR,-1.8333,-65.3667,76.39,86,99,1.30,overcast clouds
734,734,Pedernales,DO,18.0384,-71.7440,78.94,69,10,3.24,clear sky
737,737,Acandi,CO,8.5063,-77.2778,78.66,87,39,9.22,scattered clouds


In [46]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(ideal_cities_df))

(array([  8,  32,  55, 119]), array([2, 2, 2, 2]))

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
ideal_cities_df = ideal_cities_df.dropna()
clean_df = ideal_cities_df

In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = "name"
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,name
10,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,name
12,Hilo,US,78.24,few clouds,19.7297,-155.0900,name
15,Dakar,SN,72.48,broken clouds,14.6937,-17.4441,name
19,Saint-Philippe,RE,78.39,light rain,-21.3585,55.7679,name
27,Soyo,AO,78.55,overcast clouds,-6.1349,12.3689,name
30,Cayenne,GF,80.10,light rain,4.9333,-52.3333,name
31,Saint George,US,73.87,clear sky,37.1041,-113.5841,name
36,East London,ZA,73.27,overcast clouds,-33.0153,27.9116,name
37,Mana,GF,76.24,scattered clouds,5.6592,-53.7767,name


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        

Hotel not found... skipping.201
Hotel not found... skipping.238
Hotel not found... skipping.339
Hotel not found... skipping.437
Hotel not found... skipping.467
Hotel not found... skipping.478
Hotel not found... skipping.483
Hotel not found... skipping.501
Hotel not found... skipping.528
Hotel not found... skipping.551
Hotel not found... skipping.580
Hotel not found... skipping.587
Hotel not found... skipping.596
Hotel not found... skipping.610
Hotel not found... skipping.669
Hotel not found... skipping.731
Hotel not found... skipping.733


In [57]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,People ThankYou
10,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
12,Hilo,US,78.24,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Dakar,SN,72.48,broken clouds,14.6937,-17.4441,Redface Studios Meublés
19,Saint-Philippe,RE,78.39,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
732,Richards Bay,ZA,75.79,broken clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
733,Maraa,BR,76.39,overcast clouds,-1.8333,-65.3667,name
734,Pedernales,DO,78.94,clear sky,18.0384,-71.7440,Hotel Costa Marina
737,Acandi,CO,78.66,scattered clouds,8.5063,-77.2778,Hotel Patio Bonito


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [60]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))